In [1]:
import pandas as pd
import sqlite3
import sqlalchemy
import glob

In [2]:
main_file = "data.parquet"
df = pd.read_parquet(main_file, engine='auto')

# divide collumn tpep_dropoff_datatime into date and time respectively
df[['tpep_dropoff_date', 'tpep_dropoff_time']] = df['tpep_dropoff_datetime'].astype(str).str.split(' ', 1, expand = True)

# delete the excessive tpep_dropoff_time collumn
del df['tpep_dropoff_time']

# divide the file based on tpep_dropoff_date:
df.to_parquet('./partitioned', partition_cols=['tpep_dropoff_date'])

In [3]:
# view how the main file was read
#df

In [4]:
# Select which files will be reuploaded based on their dropoff date:
part_files = "./partitioned/*2020-01*/*"

# Re-read files, but only with expected tpep_dropoff_datetime:
newdf = pd.read_parquet(glob.glob(part_files), engine='auto')

In [5]:
# Uncomment to view how the files have been read after eliminating anomalies
#newdf

In [6]:
#compare the number of rows that we started with, to the number of rows that were re-read:
print("Original rows: ", len(df.index))
print("Re-read rows:  ", len(newdf.index))

Original rows:  6405008
Re-read rows:   6402165


In [7]:
engine = sqlalchemy.create_engine('sqlite:///DataBase.db')
newdf.to_sql('table', engine, if_exists = 'append', index = False, chunksize = 1000, method = None)

6402165